# Landau-Zener Model Simulation in IBMQ

### Abstract
This notebook presents the time evolution simulation of a qubit according to the Landau Zener (LZ) Hamiltonian on a real IBM Quantum Computer. Time evolution is discretized by performing small finite evolutions and the experimental probabilities of the $|0\rangle$ state are measured for a given qubit as a function of the simulation time.

### Introduction


### Useful Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg
import csv
import qiskit
from qiskit import *
from qiskit.tools.monitor import job_monitor
from IPython.display import clear_output
import itertools

### IBMQ Setup

In [2]:
IBMQ.load_account(); # Load IBM Qiskit Account
provider = IBMQ.get_provider('ibm-q') # IBM provider

### LZ Hamiltonian

The LZ Hamiltonian expressed in terms of the $\{|0\rangle,|1\rangle\}$ basis is

$$\hat{H} = -\frac{\varepsilon(t)}{2}\hat{\sigma}_z - \frac{\Delta}{2}\hat{\sigma}_x$$

where $\varepsilon(t) = t/t_a$ and $\Delta$ is the energy bandgap at the avoided level crossing at $t=0$. The following figure shows the inverse of the instantaneous energy gap of this Hamiltonian in time and inset are the two energy levels of the system. The diabatic states $|0\rangle$ and $|1\rangle$ are eigenstates of the Hamiltonian at $t\to\pm\infty$.


<img src="KZM_1.png" alt="Drawing" style="width: 400px;"/>


The adiabatic theorem states that while the energy bandgap is big enough, the system will approximately remain on the instantaneous eigenstate of the Hamiltonian. Near the avoided level crossing at $t=0$, the system undergoes significant LZ transitions as a consequence of the inverse of the energy gap reaching a maximum.

In [3]:
# LZ Hamiltonian
def H(t, ta, Delta):
    return -0.5 * np.array([[t / ta, Delta], [Delta, -t / ta]])

### Time Evolution Simulation
In order to approximate the time evolution of a qubit according to the LZ Hamiltonian using a digital quantum computer, it is necessary to perform a discretization of the time evolution operator. Dividing the total time interval $t_f-t_i$ in $N_t$ points, the Hamiltonian can be evaluated in each point and the qubit is evolved as in a time independent case for a small interval $dt = (t_f-t_i)/N_t$ (i.e. the size of the steps).

<img src="KZM_2.png" alt="Drawing" style="width: 600px;"/>

Each exponential operator from the time independent cases can be included to the quantum circuit to simulate the time evolution as a product of unitary gates

$$\hat{U}(t,t_i) \approx \prod_{k=0}^{N-1}e^{-i\hat{H}_kdt},$$

where $\hat{H}_k = \hat{H}(t_i + kdt).$

### Measurement Calibration
Readout errors are one of the main sources of noise in the experimental probabilities obtained. The results can be corrected using a calibration matrix $\mathbf{A}$ which can be obtained from the backend properties:

* prob_meas1_prep0: probability of preparing state $|0\rangle$ and measuring state $|1\rangle$.
* prob_meas0_prep1: probability of preparing state $|1\rangle$ and measuring state $|0\rangle$.

Given a vector of noisy probabilities $\mathbf{P}_{\text{noisy}}$ representing the experimental readouts and a vector of ideal probabilities $\mathbf{P}_{\text{ideal}}$

$$\mathbf{P}_{\text{noisy}} = \mathbf{A}\mathbf{P}_{\text{ideal}}$$

where

$$\mathbf{A} = \pmatrix{1-\text{prob_meas1_prep0} & \text{prob_meas0_prep1}\\ \text{prob_meas1_prep0} & 1-\text{prob_meas0_prep1}}$$

finally, ideal probabilities can be recovered by inverting the calibration matrix

$$\mathbf{P}_{\text{ideal}} = \mathbf{A}^{-1}\mathbf{P}_{\text{noisy}}$$

### Useful Functions

In [6]:
def initGroundState(ti, ta, Delta):
    """
    initGroundState computes the instantaneous lower energy eigenstate of the LZ Hamiltonian
    
    Parameters
    ----------
    ti            Initial simulation time
    ta            Annealing time
    Delta         Energy gap at t=0
    
    Returns
    -------
    initState     Initial ground state
    
    """
    
    Htemp = H(ti, ta = ta, Delta = Delta)
        
    eigVal = scipy.linalg.eigvals(Htemp)
    eigVec = scipy.linalg.eig(Htemp, left=False, right=True)[1]

    indBaseState = np.argmin(eigVal)
    initState = eigVec[:, indBaseState]
    
    return initState

def counts2probability(counts, target):
    """
    counts2probability computes the probabilities for a given target qubit
    
    Parameters
    ----------
    counts        IBM Results count structure
    target        Target qubit
    
    Returns
    -------
    probability   Experimental probability vector
    
    """
    
    global shots, props
    n_qubits =  len(props.qubits)
    
    lst = np.asarray(list(itertools.product([0, 1], repeat=n_qubits)))
    ind = np.where(lst[:, n_qubits - target - 1] == 0)[0]
    
    shots_zero = 0
    
    for num in ind:
        if isinstance(counts.get(str(hex(num))), int):
            shots_zero = shots_zero + counts.get(str(hex(num)))
    
    probability = (1 / shots) * np.array([shots_zero, shots - shots_zero])
    
    return probability

def getCalMatrix(backend, target):
    """
    getCalMatrix obtains de calibration matrix for the target qubit of the specified backend
    
    Parameters
    ----------
    backend       IBM backend
    target        Target qubit
    
    Returns
    -------
    calMatrix     Calibration matrix
    
    """
    
    global props
    
    qi = props.qubits[target]
            
    # Set Calibration Matrices
    p01 = qi[5].value
    p10 = qi[6].value
    calMatrix = np.array([[1-p10, p01], [p10, 1 - p01]])
    
    return calMatrix

def calibrateProbability(backend, target, probability):
    """
    calibrateProbability calculates the calibrated probability using the calibration matrix
    
    Parameters
    ----------
    backend            IBM backend
    target             Target qubit
    probability        Experimental probability vector
    
    Returns
    -------
    calProbability     Calibrated probability
    
    """
    
    calProbability = getCalMatrix(backend, target) @ probability
    
    return calProbability

## Run Experiment

In [7]:
# IBM backend and properties
backend_name = 'ibmq_quito'

backend = provider.get_backend(backend_name)
# backend = Aer.get_backend('qasm_simulator')
props = backend.properties()

In [8]:
# Target qubit
target = 2

# Shots per datapoint
shots = 5000

In [9]:
# LZ Hamiltonian parameters
ta = 1      # Annealing time
Delta = 1   # Energy gap

# Time evolution parameters
ti = 0      
tf = 10     
Nt = 5     

dt = (tf - ti) / Nt # Time step
t = np.linspace(ti, tf, Nt) # Time vector

In [ ]:
# Initial eigenstate calculation
initState = initGroundState(ti, ta, Delta)

# Initialize probability lists
Praw = []
Pcal = []
    
for N in range(Nt):
    # Show progress
    clear_output(wait=True)
    print(N+1, '/', Nt)
    
    # Create quantum circuit
    qr = QuantumRegister(len(props.qubits), name = 'q')
    cr = ClassicalRegister(len(props.qubits), name = 'c')
    circuit = QuantumCircuit(qr, cr)
    
    # Qubit Initial States
    circuit.initialize(initState, target)
    circuit.barrier()
    
    # Add Time Evolution Gates
    for k in range(N):
        tk = ti + k * dt
        Hk = H(tk, ta = ta, Delta = Delta)
        dU = scipy.linalg.expm(-1j * Hk * dt)
        circuit.unitary(dU, qr[target])
        circuit.barrier()
        
    # Add measurement
    circuit.measure(qr, cr)
    
    # Run on IBM Quantum
    job = execute(circuit, backend=backend, shots=shots)
    job_monitor(job)
    
    # Experiment Results
    result = job.result()
    counts = result.data().get('counts')
    
    # Raw experimental probability
    Pexp = counts2probability(counts, target)
    Praw.append(Pexp)
    
    # Calibrated experimental probability
    P = calibrateProbability(backend, target, probability=Pexp)
    Pcal.append(P)

1 / 5
Job Status: job is queued (14)    